In [2]:
filename = "OR_ABI-L2-TPWF-M6_G16_s20223000300207_e20223000309515_c20223000311343.nc"

start = filename[filename.find("_s") + 2 : filename.find("_e")]
# Get year
year = int(start[0:4])
# Get julian day
julian_day = int(start[4:7])

# Time (UTC) as string
hour_utc = (start[7:11])

print(f">>>>>>>>>date_save {year}")
print(f">>>>>>>>> hour_utc {hour_utc}")
print(f">>>>>>>>>julian_day  {julian_day}")

>>>>>>>>>date_save 2022
>>>>>>>>> hour_utc 0300
>>>>>>>>>julian_day  300


In [1]:
import pendulum
datetime_save = "20221027 300"
pendulum.from_format(datetime_save, "YYYYMMDD HHmm")

ValueError: hour must be in 0..23

In [1]:
# https://github.com/prefeitura-rio/pipelines/blob/master/pipelines/utils/utils.py#L369
import basedosdados as bd
import numpy as np
import pandas as pd
import pendulum
import s3fs


from satelite.constants import (
    constants as satelite_constants,
) #pipelines.rj_cor.meteorologia.

In [2]:
def get_path_files_s3(variabel):
    s3_fs = s3fs.S3FileSystem(anon=True)

    # Get all files of GOES-16 data (multiband format) at this hour
    path_files = s3_fs.find(f"noaa-goes16/ABI-L2-{variabel}/")
    return path_files

variavel_rr = satelite_constants.VARIAVEL_RR.value
variavel_tpw = satelite_constants.VARIAVEL_TPW.value
variavel_cmip = satelite_constants.VARIAVEL_cmip.value
# cmip_files = get_all_files_s3(variavel=variavel_rr)
# cmip_files

In [3]:
# get_all_path_files():
all_files = []
for var in [variavel_rr, variavel_tpw, variavel_cmip]:
    print(var)
    all_files = all_files + get_path_files_s3(var)

# keep only the start scan datetime
print(len(all_files))
ref_dates = list(set([file[file.find("_s") + 1: file.find("_e")] for file in all_files]))
print(len(ref_dates))

RRQPEF
TPWF
CMIPF


EndpointConnectionError: Could not connect to the endpoint URL: "https://noaa-goes16.s3.amazonaws.com/?list-type=2&prefix=ABI-L2-CMIPF%2F&delimiter=&continuation-token=1C32vNDW3YKleSwOEY6pMb6AdRaTOeDflzk3oEuqx9UQmxGoJBK0IjXgpr6FVJLBymddxEV1GLxiD4mOSq7ETyfLUChUlCCQyiFHF5SH49a7l3KOotsF05Nk5IBV5XZnTdslDA3f9B0RP4z0%2Fn4zAxSFBcTai19LMxaLSHx3QLmc%3D&encoding-type=url"

In [24]:
all_files

['noaa-goes16/ABI-L2-RRQPEF/2019',
 'noaa-goes16/ABI-L2-RRQPEF/2020',
 'noaa-goes16/ABI-L2-RRQPEF/2021',
 'noaa-goes16/ABI-L2-RRQPEF/2022',
 'noaa-goes16/ABI-L2-TPWF/2019',
 'noaa-goes16/ABI-L2-TPWF/2020',
 'noaa-goes16/ABI-L2-TPWF/2021',
 'noaa-goes16/ABI-L2-TPWF/2022',
 'noaa-goes16/ABI-L2-CMIPF/2000',
 'noaa-goes16/ABI-L2-CMIPF/2017',
 'noaa-goes16/ABI-L2-CMIPF/2018',
 'noaa-goes16/ABI-L2-CMIPF/2019',
 'noaa-goes16/ABI-L2-CMIPF/2020',
 'noaa-goes16/ABI-L2-CMIPF/2021',
 'noaa-goes16/ABI-L2-CMIPF/2022']

https://stackoverflow.com/questions/45429556/how-list-amazon-s3-bucket-contents-by-modified-date

In [51]:



def get_storage_blobs(dataset_id: str, table_id: str) -> list:
    """
    Get all blobs from a table in a dataset.
    """

    storage = bd.Storage(dataset_id=dataset_id, table_id=table_id)

    return list(
        storage.client["storage_staging"]
        .bucket(storage.bucket_name)
        .list_blobs(prefix=f"staging/{storage.dataset_id}/{storage.table_id}/")
    )

def get_storage_blobs_adaptado(bucket_name: str, dataset_id: str, table_id: str) -> list:
    """
    Get all blobs from a table in a dataset.
    """

    storage = bd.Storage(dataset_id=dataset_id, table_id=table_id)

    return list(
        storage.client["storage_staging"]
        .bucket(bucket_name)
        .list_blobs(prefix=f"staging/{dataset_id}/{table_id}/")
    )


def parser_blobs_to_partition_dict(blobs: list) -> dict:
    """
    Extracts the partition information from the blobs.
    """

    partitions_dict = {}
    partitions_list = []

    for blob in blobs:
        for folder in blob.name.split("/"):
            if "=" in folder:
                key = folder.split("=")[0]
                value = folder.split("=")[1]
                try:
                    partitions_dict[key].append(value)
                except KeyError:
                    partitions_dict[key] = [value]
    return partitions_dict, partitions_list



In [52]:
def obter_datas_particoes(list_storage_blobs):
    # Para o caso de dados de satélite que temos partição de ano, mes, dia e hora
    dates_blob = []
    for blob in list_storage_blobs:
        for folder in blob.name.split("/"):
            #print(folder)
            if "=" in folder:
                key = folder.split("=")[0]
                value = folder.split("=")[1]
                if key=='ano_particao':
                    ano = value
                elif key=='mes_particao':
                    mes = value.zfill(2)
                elif key=='data_particao':
                    dia = value[-2:].zfill(2)
                elif key=='hora':
                    hora = value.zfill(2)
        data = ano + '-' + mes + '-' + dia + ' ' + hora + ':00:00'
        dates_blob.append(data)

    return dates_blob



def datas_para_mover_datas_faltantes(bucket_name, bucket_name_antigo, table_id, table_id_antigo, dataset_id, dates_range):
    # seleciona tanto as datas faltantes pq deu algum problema na hora de mover de um bucket pro outro
    # quanto as datas faltantes pq deu problema no backfill

    list_storage_blobs_antigo = get_storage_blobs_adaptado(bucket_name=bucket_name_antigo, dataset_id=dataset_id, table_id=table_id_antigo)
    list_storage_blobs = get_storage_blobs_adaptado(bucket_name=bucket_name, dataset_id=dataset_id, table_id=table_id)

    bucket_antigo = obter_datas_particoes(list_storage_blobs_antigo)
    bucket_novo = obter_datas_particoes(list_storage_blobs)

    print(len(bucket_antigo), len(bucket_novo))

    # pegar todas as datas do bucket antigo que não estão no novo para movermos
    mover = [i for i in bucket_antigo if i not in bucket_novo]
    print('quantos mover', len(mover))

    # datas faltantes devido ao backfill
    missing_dates = [i for i in dates_range if i not in bucket_novo]
    print('quantas datas faltantes', len(missing_dates))

    return mover, missing_dates



def datas_faltantes(bucket_name, table_id, dataset_id, dates_range):
    # seleciona tanto as datas faltantes pq deu problema no backfill

    list_storage_blobs = get_storage_blobs_adaptado(bucket_name=bucket_name, dataset_id=dataset_id, table_id=table_id)

    bucket_novo = obter_datas_particoes(list_storage_blobs)

    print(len(bucket_novo))

    # datas faltantes devido ao backfill
    missing_dates = [i for i in dates_range if i not in bucket_novo]
    print('quantas datas faltantes', len(missing_dates))

    return bucket_novo, missing_dates

In [54]:
# range de datas que deveriam estar preenchidas
dates_range = list(pd.date_range(start='1/1/2020', end='1/6/2022', freq='H').astype(str))
len(dates_range)

17665

In [55]:
# bucket_name_antigo = 'rj-escritorio-dev'
# table_id_antigo = 'satelite_quantidade_agua_precipitavel'
bucket_name = 'rj-cor'
table_id = 'quantidade_agua_precipitavel_satelite'
dataset_id = 'meio_ambiente_clima'

# mover_qt, missing_dates_qt = datas_para_mover_datas_faltantes(bucket_name, bucket_name_antigo, table_id, table_id_antigo, dataset_id, dates_range)
bucket_qt, missing_dates_qt = datas_faltantes(bucket_name, table_id, dataset_id, dates_range)

19054
quantas datas faltantes 1560


In [27]:
bucket_qt[:10]

['2020-12-31 22:00:00',
 '2020-12-31 23:00:00',
 '2021-01-01 00:00:00',
 '2021-01-01 01:00:00',
 '2021-01-01 10:00:00',
 '2021-01-01 11:00:00',
 '2021-01-01 12:00:00',
 '2021-01-01 13:00:00',
 '2021-01-01 14:00:00',
 '2021-01-01 15:00:00']

In [56]:
dates_range[:5]

['2020-01-01 00:00:00',
 '2020-01-01 01:00:00',
 '2020-01-01 02:00:00',
 '2020-01-01 03:00:00',
 '2020-01-01 04:00:00']

In [57]:
# bucket_name_antigo = 'rj-escritorio-dev'
# table_id_antigo = 'satelite_taxa_precipitacao'
bucket_name = 'rj-cor'
table_id = 'taxa_precipitacao_satelite'
dataset_id = 'meio_ambiente_clima'

# mover_tx, missing_dates_tx = datas_para_mover_datas_faltantes(bucket_name, bucket_name_antigo, table_id, table_id_antigo, dataset_id, dates_range)
bucket_tx, missing_dates_tx = datas_faltantes(bucket_name, table_id, dataset_id, dates_range)

21057
quantas datas faltantes 110


In [58]:
# seleciona quais datas faltantes são comuns a ambos
missing_dates_comum = [i for i in missing_dates_tx if i in missing_dates_qt]
len(missing_dates_comum)

108

In [59]:
# datas faltantes que não são comuns
missing_dates_tx_so = [i for i in missing_dates_tx if i not in missing_dates_comum]
print(len(missing_dates_tx_so))

missing_dates_qt_so = [i for i in missing_dates_qt if i not in missing_dates_comum]
print(len(missing_dates_qt_so))

2
1452


In [60]:
# como as datas vieram das partições elas estão no timezone br, precisamos converter pro tz UTC para rodar no backfill
def converte_tz(data):
    return pendulum.from_format(data, 'YYYY-MM-DD HH:mm:SS', tz='America/Sao_Paulo').in_tz('UTC').to_datetime_string()


In [61]:
missing_dates_qt_so_tz = [converte_tz(i) for i in missing_dates_qt_so]
missing_dates_tx_so_tz = [converte_tz(i) for i in missing_dates_tx_so]
missing_dates_comum_tz = [converte_tz(i) for i in missing_dates_comum]

# print(missing_dates_tx_so_tz)
# print(missing_dates_qt_so_tz)

In [62]:
len(missing_dates_comum_tz)

108

In [67]:
missing_dates_comum_tz

['2020-06-03 17:00:00',
 '2020-06-03 18:00:00',
 '2020-06-03 19:00:00',
 '2020-06-03 20:00:00',
 '2020-06-03 21:00:00',
 '2020-06-03 22:00:00',
 '2020-06-03 23:00:00',
 '2020-06-04 00:00:00',
 '2020-07-26 21:00:00',
 '2020-07-26 22:00:00',
 '2020-07-26 23:00:00',
 '2020-07-27 00:00:00',
 '2020-07-27 01:00:00',
 '2020-07-27 02:00:00',
 '2020-08-11 16:00:00',
 '2020-08-11 17:00:00',
 '2020-08-11 18:00:00',
 '2021-01-21 20:00:00',
 '2021-01-21 21:00:00',
 '2021-03-16 15:00:00',
 '2021-03-23 14:00:00',
 '2021-03-23 15:00:00',
 '2021-03-23 16:00:00',
 '2021-03-23 17:00:00',
 '2021-03-23 18:00:00',
 '2021-03-23 19:00:00',
 '2021-03-23 20:00:00',
 '2021-04-29 21:00:00',
 '2021-04-29 22:00:00',
 '2021-06-15 12:00:00',
 '2021-06-29 12:00:00',
 '2021-06-29 13:00:00',
 '2021-06-29 14:00:00',
 '2021-06-29 15:00:00',
 '2021-06-29 16:00:00',
 '2021-06-29 17:00:00',
 '2021-06-29 18:00:00',
 '2021-07-21 17:00:00',
 '2021-09-18 03:00:00',
 '2021-09-18 04:00:00',
 '2021-09-18 05:00:00',
 '2021-09-18 06:

In [66]:
missing_dates_qt_so_tz[1000:]

['2020-09-11 10:00:00',
 '2020-09-11 16:00:00',
 '2020-09-11 17:00:00',
 '2020-09-12 00:00:00',
 '2020-09-12 10:00:00',
 '2020-09-12 16:00:00',
 '2020-09-12 17:00:00',
 '2020-09-12 18:00:00',
 '2020-09-13 06:00:00',
 '2020-09-13 14:00:00',
 '2020-09-13 19:00:00',
 '2020-09-14 13:00:00',
 '2020-09-14 17:00:00',
 '2020-09-14 18:00:00',
 '2020-09-14 19:00:00',
 '2020-09-15 01:00:00',
 '2020-09-15 07:00:00',
 '2020-09-15 09:00:00',
 '2020-09-15 13:00:00',
 '2020-09-15 14:00:00',
 '2020-09-15 15:00:00',
 '2020-09-15 20:00:00',
 '2020-09-16 09:00:00',
 '2020-09-16 11:00:00',
 '2020-09-16 16:00:00',
 '2020-09-16 21:00:00',
 '2020-09-17 09:00:00',
 '2020-09-17 13:00:00',
 '2020-09-17 14:00:00',
 '2020-09-17 21:00:00',
 '2020-09-18 02:00:00',
 '2020-09-18 11:00:00',
 '2020-09-18 20:00:00',
 '2020-09-18 21:00:00',
 '2020-09-19 11:00:00',
 '2020-09-19 15:00:00',
 '2020-09-20 02:00:00',
 '2020-09-20 07:00:00',
 '2020-09-20 08:00:00',
 '2020-09-20 13:00:00',
 '2020-09-20 17:00:00',
 '2020-09-20 22:

In [ ]:
missing_dates_comum_tz[2000:]

In [99]:
import datetime as dt
current_time = '2021-07-21 18:00:00'
current_time = '2020-03-20 16:00:00'
current_time = '2020-06-03 23:00:00'
# current_time =  '2020-03-21 11:00:00'
dt.datetime.strptime(current_time, "%Y-%m-%d %H:%M:%S").strftime("%j")

'155'

In [50]:
pd.DataFrame(missing_dates_comum_tz).to_csv("teste_apagar*.csv", index=False)